In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from IPython.core.display import display, HTML

from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from collections import Counter
pd.options.display.max_seq_items = None

In [2]:
df = pd.read_csv("../../data/features_model/df_story_features_model_whole.csv")


C:\Users\Justin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
from bs4 import BeautifulSoup
display(df.head())

,Unnamed: 0,Unnamed: 0.1,ClapCount_Card,ClapCount_Story,Company,CompanyURL,PostID,PublishedDate,ReadingTime,ResponseNum_Card,ResponseNum_Story,VoterCount,isPaywall,TagNum,isPublication,AuthorArticlesClap_mean,AuthorArticlesClap_median,AuthorArticlesClap_count,AuthorArticlesClap_sum,AuthorArticlesResponse_mean,AuthorArticlesResponse_median,AuthorArticlesResponse_count,AuthorArticlesResponse_sum,AuthorArticlesVoter_mean,AuthorArticlesVoter_median,AuthorArticlesVoter_count,AuthorArticlesVoter_sum,PublicationClap_mean,PublicationClap_median,PublicationClap_count,PublicationClap_sum,PublicationReadingTime_mean,PublicationReadingTime_median,PublicationReadingTime_count,PublicationReadingTime_sum,PublicationVoter_mean,PublicationVoter_median,PublicationVoter_count,PublicationVoter_sum,PublicationisPaywall_mean,PublicationisPaywall_median,PublicationisPaywall_count,PublicationisPaywall_sum,TagUseSum,TagUseMean,TagUseMedian,TagClapSum,TagClapMean,TagClapMedian,SentimentPolarity,SentimentSubjectivity,WordNum,TextSyllableNum,TextLexiconNum,TextSentenceNum,ReadabilityFleschEase,ReadabilitySMOG,ReadabilityFleschKincaid,ReadabilityColemanLiau,ReadabilityARI,ReadabilityDaleChall,ReadabilityDifficultWordsNum,ReadabilityLinsearWriteFormula,ReadabilityGunningFog,ReadabilityReadingTime,ReadabilityConsensus,HasFeaturedImage,CodeInlineNum,CodeBlockNum,CodeBlockLengthSum,CodeBlockLengthMedian,CodeBlockLengthMean,CodeBlockLengthStd,CodeBlockLengthMin,CodeBlockLengthMax,ListOlNum,ListOlLength,ListOlSum,ListOlMedian,ListOlMean,ListOlStd,ListOlMin,ListOlMax,ListUlNum,ListUlLength,ListUlSum,ListUlMedian,ListUlMean,ListUlStd,ListUlMin,ListUlMax,ImgNum,LinkNum,HlightNum,ParagraphNum,ItalicNum,BoldNum
0,0,0,2,2.0,0,0,f5adff36e2cc,2020-02-04,4,0,0,2.0,0,3,0,20.166667,13.5,6,121.0,4.500000,4.0,6,27.0,0.166667,0.0,6,1.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,10739.0,3579.666667,25.0,1017888.0,339296.0,813.0,0.172666,0.462687,824,1196,823,23,43.60,15.6,16.1,10.63,19.4,7.68,118,20.000000,17.38,58.72,16.0,1,7,0,0,0.0,0.000000,0.000000,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,13,2,0,19,1,2
1,1,1,198,198.0,The Writing Cooperative,https://writingcooperative.com?source=tag_arch...,35c2918e89ba,2018-12-31,5,1 response,1,25.0,1,5,1,246.736842,85.0,57,14064.0,32.631579,6.0,57,1860.0,0.280702,0.0,57,16.0,335.753822,133.0,5037.0,1691192.0,4.708954,4.0,5037.0,23719.0,40.124876,15.0,5037.0,202109.0,0.515585,1.0,5037.0,2597.0,180517.0,36103.400000,0.0,29493305.0,5898661.0,0.0,0.171449,0.453979,1099,1604,1098,46,55.68,12.8,11.4,11.49,14.3,7.21,166,12.200000,12.66,81.54,13.0,1,0,0,0,0.0,0.000000,0.000000,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,11,15,0,26,5,2
2,2,2,987,987.0,freeCodeCamp.org,https://medium.com/free-code-camp?source=tag_a...,35c856968991,2018-12-07,6,7 responses,7,182.0,0,5,1,246.736842,85.0,57,14064.0,32.631579,6.0,57,1860.0,0.280702,0.0,57,16.0,2283.122672,675.0,4296.0,9808295.0,7.620112,6.0,4296.0,32736.0,489.151304,140.0,4296.0,2101394.0,0.043762,0.0,4296.0,188.0,533926.0,106785.200000,116003.0,66535017.0,13307003.4,16808509.0,0.122516,0.502721,1324,1999,1320,71,61.06,11.2,9.4,11.49,12.2,6.96,201,9.000000,10.20,100.20,9.0,1,33,2,5,2.5,2.500000,0.500000,2,3,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,4,8,0,29,0,7
3,3,3,523,523.0,Towards Data Science,https://towardsdatascience.com?source=tag_arch...,22232699259f,2018-12-31,4,1 response,1,159.0,1,5,1,789.181818,523.0,11,8681.0,191.545455,145.0,11,2107.0,1.000000,1.0,11,11.0,389.547934,128.0,21738.0,8467993.0,7.076042,6.0,21738.0,153819.0,82.347870,25.0,21738.0,1790078.0,0.526267,1.0,21738.0,11440.0,180722.0,36144.400000,27915.0,28447893.0,5689578.6,2329840.0,0.109809,0.437326,608,963,602,24,46.00,14.2,13.1,12.13,15.6,7.77,110,10.833333,13.83,45.92,14.0,1,7,1,1,1.0,1.000000,0.000000,1,1,1,[34],34,34.0,34.0,0.0,34,34,0,0,0,0.0,0.0,0.0,0,0,37,6,0,21,0,2
4,4,4,264,264.0,The Startup,https://medium.com/swlh?source=tag_archive----...,5e3f9b53cbf4,2019-12-31,

# Generate binary dependent variable (for logit / SVM)

In [6]:
# Median split: Doesnt work if the median claps is 0
# df['isPopular'] = (df['ClapCount_Story'] > df['ClapCount_Story'].median()).astype(int)

# Below or above the top 25% of claps
threshold = 0.30
df['isPopular'] = (df['ClapCount_Story'] > df.sort_values(by=['ClapCount_Story'], ascending=False).iloc[int(len(df)*threshold)]['ClapCount_Story'])

# Split train / test (80/20)

In [43]:
sample = df.sample(500000)
sample = df

mask = np.random.rand(len(sample)) < 0.8
train = sample[mask]
test = sample[~mask]

In [15]:
print("Length of dataset is", len(df))
print("Median clap is", train['ClapCount_Story'].median())

Length of dataset is 5972614
Median clap is 1.0


In [ ]:
xFeatures = ['isPaywall', 'isPublication', 'WordNum', 'SentimentPolarity', 'SentimentSubjectivity',  "TextSyllableNum",
          "TextLexiconNum", "TextSentenceNum", "ReadabilityFleschEase", "ReadabilitySMOG", "ReadabilityFleschKincaid", "ReadabilityColemanLiau", "ReadabilityARI", "ReadabilityDaleChall", "ReadabilityDifficultWordsNum", "ReadabilityLinsearWriteFormula", "ReadabilityGunningFog", "ReadabilityReadingTime", "ReadabilityConsensus", 
        'ReadingTime', 'HasFeaturedImage', 'CodeInlineNum', 'TagNum',
        'CodeBlockNum', 'CodeBlockLengthSum', 'CodeBlockLengthMedian', 'CodeBlockLengthMean',
        'CodeBlockLengthStd', 'CodeBlockLengthMin', 'CodeBlockLengthMax',
        'ListOlNum', 'ListOlSum', 'ListOlMedian', 'ListOlMean', 'ListOlStd', 'ListOlMin', 'ListOlMax', 
        'ListUlNum', 'ListUlSum', 'ListUlMedian', 'ListUlMean', 'ListUlStd', 'ListUlMin', 'ListUlMax', 
        'ImgNum', 'LinkNum',
        'HlightNum', 'ParagraphNum', 'ItalicNum', 'BoldNum', 
        'AuthorArticlesClap_mean','AuthorArticlesClap_median','AuthorArticlesClap_count','AuthorArticlesClap_sum',
        'AuthorArticlesResponse_mean','AuthorArticlesResponse_median','AuthorArticlesResponse_sum',
        'AuthorArticlesVoter_mean','AuthorArticlesVoter_median','AuthorArticlesVoter_sum',
        
        'TagUseSum', 'TagUseMean', 'TagUseMedian', 
        'TagClapSum','TagClapMean', 'TagClapMedian',  
        'PublicationClap_mean','PublicationClap_median','PublicationClap_count','PublicationClap_sum',
        'PublicationReadingTime_mean','PublicationReadingTime_median','PublicationReadingTime_sum',
        'PublicationVoter_mean','PublicationVoter_median','PublicationVoter_sum',
        'PublicationisPaywall_mean','PublicationisPaywall_median','PublicationisPaywall_sum',]

yFeatures = 'ClapCount_Story' # For linreg
# yFeatures = 'isPopular' # For logreg

xTrain = train[xFeatures]
yTrain = train[yFeatures]

xTest = test[xFeatures]
yTest = test[yFeatures]

In [44]:
features = ["isPaywall","isPublication",
"SentimentPolarity","SentimentSubjectivity",
"WordNum","TextSyllableNum","TextLexiconNum","TextSentenceNum",
"ReadabilityFleschEase","ReadabilitySMOG","ReadabilityFleschKincaid","ReadabilityColemanLiau","ReadabilityARI","ReadabilityDaleChall","ReadabilityDifficultWordsNum","ReadabilityLinsearWriteFormula","ReadabilityGunningFog","ReadabilityReadingTime","ReadabilityConsensus",
"ReadingTime","HasFeaturedImage","CodeInlineNum","TagNum","CodeBlockNum",
"CodeBlockLengthSum","CodeBlockLengthMedian","CodeBlockLengthMean","CodeBlockLengthStd","CodeBlockLengthMin","CodeBlockLengthMax",
"ListOlNum","ListOlSum","ListOlMedian","ListOlMean","ListOlStd","ListOlMin","ListOlMax",
"ListUlNum","ListUlSum","ListUlMedian","ListUlMean","ListUlStd","ListUlMin","ListUlMax","ImgNum","LinkNum","HlightNum",
"ParagraphNum","ItalicNum","BoldNum",
# "AuthorArticlesClap_mean","AuthorArticlesClap_median","AuthorArticlesClap_count","AuthorArticlesClap_sum",
"AuthorArticlesClap_count",
"AuthorArticlesResponse_median","AuthorArticlesResponse_sum",
"AuthorArticlesResponse_mean","AuthorArticlesVoter_mean","AuthorArticlesVoter_median","AuthorArticlesVoter_sum",
"TagUseSum","TagUseMean","TagUseMedian","TagClapSum","TagClapMean","TagClapMedian",
"PublicationClap_mean","PublicationClap_median",
"PublicationClap_count","PublicationClap_sum","PublicationReadingTime_mean","PublicationReadingTime_median","PublicationReadingTime_sum","PublicationVoter_mean","PublicationVoter_median","PublicationVoter_sum","PublicationisPaywall_mean","PublicationisPaywall_median","PublicationisPaywall_sum"

]
print(len(features))

# depvar = 'ClapCount_Story'
depvar = 'isPopular'


xTrain = train[features]
yTrain = train[depvar]

xTest = test[features]
yTest = test[depvar]

76


# Check NA columns

In [ ]:
# pd.set_option('display.max_rows', 1000)
# display((df[df['TagNum']==0]))
# display(xTrain.isna().any())

# Generate model

In [30]:
reg = LinearRegression(normalize=True).fit(xTrain, yTrain)

#sklearn preprocess normalize

In [31]:
reg.score(xTrain,yTrain)

0.2539417278309367

# statsmodel

In [22]:
mod = sm.OLS(yTrain,xTrain.astype(float))
fii = mod.fit()

print(dir(fii))
print(p_values[p_values<0.05].sort_values(ascending=True))
print(fii.summary())

['HC0_se', 'HC1_se', 'HC2_se', 'HC3_se', '_HCCM', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cache', '_data_attr', '_get_robustcov_results', '_is_nested', '_wexog_singular_values', 'aic', 'bic', 'bse', 'centered_tss', 'compare_f_test', 'compare_lm_test', 'compare_lr_test', 'condition_number', 'conf_int', 'conf_int_el', 'cov_HC0', 'cov_HC1', 'cov_HC2', 'cov_HC3', 'cov_kwds', 'cov_params', 'cov_type', 'df_model', 'df_resid', 'eigenvals', 'el_test', 'ess', 'f_pvalue', 'f_test', 'fittedvalues', 'fvalue', 'get_influence', 'get_prediction', 'get_robustcov_results', 'initialize', 'k_constant', 'llf', 'load', 'model', 'mse_model', 'mse_resid', 'mse_total', 'nobs', 'normalized_cov_params', 'outlie

NameError: name 'p_values' is not defined

# Logistic Regression

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
logit = LogisticRegression()
logit.fit(xTrain,yTrain)
logit.score(xTrain,yTrain)

C:\Users\Justin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

0.7941471266781145

In [34]:
# Get AUC
from sklearn import metrics

yPreds = logit.predict(xTrain)
print("Predict", Counter(yPreds))
# print(type(yPreds))
print("Truth", Counter(yTrain))

# print(yPreds)
fpr, tpr, thresholds = metrics.roc_curve(yTrain, yPreds)
print(metrics.auc(fpr, tpr))

Predict Counter({False: 342480, True: 57597})
Truth Counter({False: 281249, True: 118828})
0.6791291670593362


# SVM

In [45]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
# Always scale the input. The most convenient way is to use a pipeline.
clf = make_pipeline(StandardScaler(),
                    SGDClassifier(max_iter=1000, tol=1e-3))
clf.fit(xTrain, yTrain)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('sgdclassifier',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000, n_iter_no_change=5, n_jobs=None,
                               penalty='l2', power_t=0.5, random_state=None,
                               shuffle=True, tol=0.001, validation_fraction=0.1,
                               verbose=0, warm_start=False))],
         verbose=False)

In [47]:
clf.score(xTrain, yTrain)
print(dir(clf))
# def f_importances(coef, names):
#     imp = coef
#     imp,names = zip(*sorted(zip(imp,names)))
#     plt.barh(range(len(names)), imp, align='center')
#     plt.yticks(range(len(names)), names)
#     plt.show()

yPreds = clf.predict(xTrain)
print("Predict", Counter(yPreds))
# print(type(yPreds))
print("Truth", Counter(yTrain))

# print(yPreds)
fpr, tpr, thresholds = metrics.roc_curve(yTrain, yPreds)
print(metrics.auc(fpr, tpr))



0.8445889537836837

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_estimator_type', '_final_estimator', '_fit', '_get_param_names', '_get_params', '_get_tags', '_inverse_transform', '_iter', '_log_message', '_pairwise', '_replace_estimator', '_required_parameters', '_set_params', '_transform', '_validate_names', '_validate_steps', 'classes_', 'decision_function', 'fit', 'fit_predict', 'fit_transform', 'get_params', 'inverse_transform', 'memory', 'named_steps', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params', 'steps', 'transform', 'verbose']
Predict Counter({False: 3883526, True: 894635})
Truth Co